# Modelo de Recurrent Neural Network

# Importaciones necesarias

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from torchviz import make_dot

In [21]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x))) 
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Se crea el Modelo de CNN

In [22]:
model = CNN()

Se carga el dataset de training y de test para entrenar el modelo

In [35]:
# se cargan todas las imagenes en la carpeta PetImages y luego se dividen en train y test

transform = transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor()])
dataset = torchvision.datasets.ImageFolder(root='./PetImages', transform=transform)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)



Se entrena el modelo para que pueda hacer las predicciones

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Clean gradient before new batch
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Call backward propagation
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='./PetImages\\Dog\\10219.jpg'>

Se evalua el modelo con el dataset de test para ver su precision

In [27]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))


UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='./PetImages\\Dog\\505.jpg'>